In [ ]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

# Semi-supervised learning with GAN

- train <G,D> with the GAN framework
- use unlabeled data
- then, transform the discriminator into a classifier (10 classes)
- here, we will use the Discriminator ! 

### About the data set

In [ ]:
from keras.datasets.cifar10 import load_data

# train/val/test
from sklearn.model_selection import train_test_split

In [ ]:
(x_train, y_train), (x_test, y_test) = load_data()

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, shuffle=True, stratify=y_train, test_size=0.2)

print("x_train : {} | {}".format(x_train.shape, x_train.dtype))
print("y_train : {} | {}".format(y_train.shape, y_train.dtype))
print("x_val : {} | {}".format(x_val.shape, x_val.dtype))
print("y_val : {} | {}".format(y_val.shape, y_val.dtype))
print("x_test : {} | {}".format(x_test.shape, x_test.dtype))
print("y_test : {} | {}".format(y_test.shape, y_test.dtype))
_ = plt.hist(y_train, bins=10)

In [ ]:
fig = plt.figure(figsize=(10, 10))
for id_class in range(10):
    indices = np.where(y_train==id_class)[0]
    for i in range(10):
        plt.subplot(10, 10, id_class*10 + i + 1)
        plt.imshow(x_train[indices[i]])
plt.show()

In [ ]:
# minimal pre-processsing : scale to [-1, 1]
x_train = (x_train.astype(np.float32) / 127.5 ) - 1.
x_val = (x_val.astype(np.float32) / 127.5 ) - 1.
x_test = (x_test.astype(np.float32) / 127.5 ) - 1.

# DCGAN architecture : G and D are CNNs !

In [ ]:
# tool box 
from keras.layers import Dense, Conv2D, MaxPooling2D, AveragePooling2D, Activation, BatchNormalization, LeakyReLU, Dropout
from keras.layers import InputLayer, Input, Activation, UpSampling2D, Reshape, Flatten
from keras.models import Model, Sequential

from keras.optimizers import Adam

In [ ]:
z_dim = 100
img_height = 32
img_width = 32

### Generator

In [ ]:
input_G = Input(shape=(z_dim,))

output_G = ???

generator = Model(inputs=input_G, outputs=output_G)

generator.summary()

In [ ]:
generator.compile(optimizer='adam', loss='binary_crossentropy')

### Discriminator

In [ ]:
input_D = Input(shape=(img_height, img_width, 3))

output_D = ???


discriminator = Model(inputs=input_D, outputs=output_D)

discriminator.summary()

In [ ]:
discriminator.compile(loss='???', optimizer=????, metrics=['accuracy'])

### Combine 'G' and 'D', to train 'G'

In [ ]:
discriminator.trainable = False

gan_input = Input(shape=(z_dim,))
gan_output = discriminator(generator(gan_input))

gan = Model(inputs=gan_input, outputs=gan_output)

In [ ]:
gan.summary()

In [ ]:
gan.compile(loss='binary_crossentropy', optimizer=Adam(0.0001, beta_1=0.5), metrics=["accuracy"])

### Unsupervised 'joint' training

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
batch_size = 32
num_steps = 500

In [ ]:
batch_gen = ImageDataGenerator().flow(batch_size=batch_size,
                                      x=x_train,
                                      shuffle=True)

In [ ]:
x = batch_gen.next()
print(x.shape, x.dtype)
_ = plt.hist(x.flatten(), bins=255)

In [ ]:
i = 0

In [ ]:
for step in range(num_steps):
    ###################################### TRAIN THE DISCRIMINATOR ###############################
    discriminator.trainable = True
    # generate fake images
    random_z_vectors = np.random.normal(size=(batch_size, z_dim))
    fake_images = generator.predict(random_z_vectors)
    labels = np.ones((batch_size, 1), dtype=np.float32)
    d_loss_1 = discriminator.train_on_batch(fake_images, labels)


    # get real images
    real_images = batch_gen.next()
    labels = np.zeros((real_images.shape[0], 1), dtype=np.float32)
    d_loss_2 = discriminator.train_on_batch(real_images, labels)

    # ############################################################################################

    ###################################### TRAIN THE GENERATOR ###################################
    discriminator.trainable = False
    random_z_vectors = np.random.normal(size=(batch_size, z_dim))
    misleading_labels = np.zeros((batch_size, 1))
    g_loss = gan.train_on_batch(random_z_vectors, misleading_labels)

    ###############################################################################################
    i +=1
    if i % 50 == 0:
        print("At {} iteration : ".format(i))
        print('\tdiscriminator loss: {}'.format((d_loss_1, d_loss_2)))
        print('\tgenerator loss:  {}'.format(g_loss))
    

In [ ]:
def deprocess_images(x):
    x = (x + 1.)*127.5
    return np.clip(x, 0, 255).astype(np.uint8)

In [ ]:
random_z_vectors = np.random.normal(size=(100, z_dim))
fake_images = generator.predict(random_z_vectors)


fig = plt.figure(figsize=(10, 10))
for i in range(100):
    plt.subplot(10, 10, i+1)
    plt.imshow(deprocess_images(fake_images[i,:,:,0]))
plt.show()

## Transform D as a classifier

- you can freeze some layers of the generator

In [ ]:
discriminator.trainable = False

In [ ]:
[(layer.name, layer.trainable, layer) for layer in discriminator.layers]

In [ ]:
discriminator.input

In [ ]:
# get a layer from the discriminator
# then add you layers for the classifier

output_discriminator = ????

# add layers ? 
???

output_classifier = ???

classifier = Model(inputs=???, outputs=???)

In [ ]:
classifier.summary()

## Train the classifier with labeled data

In [ ]:
from keras.utils import to_categorical

In [ ]:
opt = Adam(lr=0.001)
classifier.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

In [ ]:
history = classifier.fit(x_train, to_categorical(y_train, num_classes=10),
                         epochs=10,
                         batch_size=32,
                         validation_data=(x_val, to_categorical(y_val, num_classes=10)),
                         shuffle=True,
                         verbose=1)